## 3 - Different Ways to Build a (Convolutional) Neural Network

In [1]:
import torch, torchvision
import matplotlib.pyplot as plt
import numpy as np
import torch.nn.functional as F



Note: Check the documentation of the layers used to learn about their usage. And note that there are many other layers in `torch.nn` module.

Conv2d: https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html

MaxPool2d: https://pytorch.org/docs/stable/generated/torch.nn.MaxPool2d.html

BatchNorm2d: https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm2d.html


There are multiple ways to create a neural network in PyTorch. The simplest one is to define layers in `__init__` method and define forward propagation logic in `forward` method (which is the way used in Lab1 and HW1.

In [2]:
class ConvNet(torch.nn.Module):
    def __init__(self):
        super().__init__()
        
        self.conv1 = torch.nn.Conv2d(in_channels=3, out_channels=32, kernel_size=5, stride=1)
        self.maxpool1 = torch.nn.MaxPool2d(kernel_size=2)
        self.batchnorm1 = torch.nn.BatchNorm2d(num_features=32)
        
        
        self.conv2 = torch.nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1)
        self.maxpool2 = torch.nn.MaxPool2d(kernel_size=2)
        self.batchnorm2 = torch.nn.BatchNorm2d(num_features=64)
        
        
        self.conv3 = torch.nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1)
        self.maxpool3 = torch.nn.MaxPool2d(kernel_size=2)
        self.batchnorm3 = torch.nn.BatchNorm2d(num_features=128)
        

        
    def forward(self, x):
        
        x = self.conv1(x)
        x = self.maxpool1(x)
        x = self.batchnorm1(x)
        x = F.relu(x)
        
        x = self.conv2(x)
        x = self.maxpool2(x)
        x = self.batchnorm2(x)
        x = F.relu(x)
        
        x = self.conv3(x)
        x = self.maxpool3(x)
        x = self.batchnorm3(x)
        
        return x
    
convnet = ConvNet()

Another way to create a neural network is using `torch.nn.Sequential` module. It takes all the layers as arguments (not as a list but separate arguments) and stacks them on top of each other, so that you won't need to define forward logic in `forward` method. But it still needs to be overridden. The behaviour of the code below is exactly the same with the one above.

Note that we even moved the activations to `self.layers` field of the module.

In [3]:
class ConvNetSequential(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = torch.nn.Sequential(
            
            #layer1
            torch.nn.Conv2d(in_channels=3, out_channels=32, kernel_size=5, stride=1),
            torch.nn.MaxPool2d(kernel_size=2),
            torch.nn.BatchNorm2d(num_features=32),
            torch.nn.ReLU(),
        
            #layer2
            torch.nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1),
            torch.nn.MaxPool2d(kernel_size=2),
            torch.nn.BatchNorm2d(num_features=64),
            torch.nn.ReLU(),
        
            #layer3
            torch.nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1),
            torch.nn.MaxPool2d(kernel_size=2),
            torch.nn.BatchNorm2d(num_features=128)
        )


        
    def forward(self, x):
        return self.layers(x)
        
convnet_sequential = ConvNetSequential()

For more complex networks, reducing the complexity is possible by defining blocks of some layers first, and using them in the network. This shows similarity with the programming paradigm that defining functions as blocks and reusing them in the code. If your network is not complex (like residual connections, dense block) this method would be an overkill but in the complex networks, this is the best way to manage complexity. Note that we used `torch.nn.Sequential` to create both `Block` and `ConvNetBlocks`. You could also use the first method but sequential method (subjectively) looks cleaner.

In [4]:
# define building block of a convolution layer, that takes the parameters as arguments
class Block(torch.nn.Module): 
    def __init__(self, in_channels, out_channels, kernel_size):
        super().__init__()
        
        self.layers = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, stride=1),
            torch.nn.MaxPool2d(kernel_size=2),
            torch.nn.BatchNorm2d(num_features=out_channels),
            torch.nn.ReLU(),
        )
        
    def forward(self, x):
        return self.layers(x)
    

# define the network using the building blocks
class ConvNetBlocks(torch.nn.Module): 
    def __init__(self):
        super().__init__()
        
        self.layers = torch.nn.Sequential(
            Block(in_channels=3,  out_channels=32,  kernel_size=5),
            Block(in_channels=32, out_channels=64,  kernel_size=3),
            Block(in_channels=64, out_channels=128, kernel_size=3),  
        )


    def forward(self, x):
        return self.layers(x)

convnet_blocks = ConvNetBlocks()

In [5]:
dummy_input = torch.randn(5, 3, 100, 100)

convnet_output             =  convnet(dummy_input)
convnet_sequential_output  =  convnet_sequential(dummy_input)
convnet_blocks_output      =  convnet_blocks(dummy_input)
print(f'Dummy input shape:              {dummy_input.shape}')
print(f'ConvNet output shape:           {convnet_output.shape}')
print(f'ConvNetSequential output shape: {convnet_sequential_output.shape}')
print(f'ConvNetBlocks output shape:     {convnet_blocks_output.shape}')

Dummy input shape:              torch.Size([5, 3, 100, 100])
ConvNet output shape:           torch.Size([5, 128, 10, 10])
ConvNetSequential output shape: torch.Size([5, 128, 10, 10])
ConvNetBlocks output shape:     torch.Size([5, 128, 10, 10])


---

## Questions to think/research on:


Question 1: `convnet` and `convnet_sequential`  are equivalent to each other while `convnet_blocks` has 1 difference from these two. What is the difference? How could the `Block` and `ConvNetBlocks` classes be changed for `convnet_blocks` to have same behavior with the other two?

(hint: it has one more operation that the other two.)

---


Question 2: After convolutional layers, it is a common practice to flatten the last convolutional layer output and use fully connected layers afterwards. How could the flattening operation be done?

---